## 기본

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm  # 진행률 표시

# 크롤링할 기업 리스트
companies = [
    "Tesla", "Google", "Nvidia", "Apple", "Volkswagen", "Mercedes",
    "Toyota", "Mastercard", "Costco", "Disney", "Netflix", "Walmart"
]

# CareerBliss 리뷰 URL
BASE_URL = "https://www.careerbliss.com/{}/reviews/?page={}"

# User-Agent 설정 (차단 방지)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 데이터 저장 리스트
data = []

# 총 작업량 설정 (기업 수 * 100개 리뷰 최대값)
total_reviews = len(companies) * 100
progress_bar = tqdm(total=total_reviews, desc="Scraping Progress", unit="review")

# 각 기업별 크롤링
for company in companies:
    print(f"\n[+] Scraping reviews for {company}...")

    company_reviews = []
    page = 1

    while len(company_reviews) < 100:  # 최대 100개 리뷰 수집
        url = BASE_URL.format(company.lower(), page)

        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"[-] Request failed for {company} on page {page}: {e}")
            break

        soup = BeautifulSoup(response.text, "html.parser")

        # 리뷰 컨테이너 찾기
        reviews = soup.find_all("div", class_="company-reviews")
        if not reviews:
            print(f"[-] No more reviews found for {company} on page {page}")
            break

        for review in reviews:
            if len(company_reviews) >= 100:  # 100개 넘어가면 종료
                break

            # 직책 (Job Title)
            job_title = review.find("a", class_="header5")
            job_title = job_title.text.strip() if job_title else "Unknown"

            # 전체 평점 (Overall Rating)
            rating_stars = review.find("div", class_="rating large-star")
            if rating_stars:
                full_stars = len(rating_stars.find_all("span", class_="full"))
                half_stars = len(rating_stars.find_all("span", class_="half"))
                overall_rating = full_stars + (0.5 * half_stars)
            else:
                overall_rating = "Unknown"

            # 세부 항목 점수
            rating_details = {}
            categories = review.find_all("span", class_="span3 header9 text-right category")
            scores = review.find_all("span", class_="span1 header8 value")

            for cat, score in zip(categories, scores):
                category_name = cat.text.strip()
                rating_score = score.text.strip()
                rating_details[category_name] = rating_score

            # 데이터 추가
            company_reviews.append({
                "Company": company,
                "Job Title": job_title,
                "Overall Rating": overall_rating,
                **rating_details
            })

            # 진행률 바 업데이트
            progress_bar.update(1)

        page += 1
        sleep_time = random.uniform(1, 3)  # 1~3초 랜덤 대기
        print(f"Sleeping for {sleep_time:.2f} seconds before next request...")
        time.sleep(sleep_time)

    data.extend(company_reviews)

# 진행률 바 종료
progress_bar.close()

# DataFrame 생성
df = pd.DataFrame(data)

# CSV 저장
df.to_csv("careerbliss_reviews_fixed.csv", index=False, encoding="utf-8-sig")
print("\n✅ Scraping complete! Data saved to 'careerbliss_reviews_fixed.csv'")



Scraping Progress:   0%|          | 0/1200 [01:36<?, ?review/s]



[+] Scraping reviews for Tesla...



Scraping Progress:   0%|          | 1/1200 [00:01<27:18,  1.37s/review]

Sleeping for 1.15 seconds before next request...



Scraping Progress:   1%|▏         | 17/1200 [00:04<04:25,  4.46review/s]

...

Sleeping for 1.70 seconds before next request...



Scraping Progress:  91%|█████████▏| 1097/1200 [04:36<00:25,  4.08review/s]

Sleeping for 1.20 seconds before next request...


Scraping Progress:  92%|█████████▏| 1100/1200 [04:38<00:25,  3.95review/s]


✅ Scraping complete! Data saved to 'careerbliss_reviews_fixed.csv'


## 기업별 수집 개수 400개로 증가

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm  # 진행률 표시

# 크롤링할 기업 리스트
companies = [
    "Tesla", "Nvidia", "Apple"
]

# CareerBliss 리뷰 URL
BASE_URL = "https://www.careerbliss.com/{}/reviews/?page={}"

# User-Agent 설정 (차단 방지)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 데이터 저장 리스트
data = []

# 총 작업량 설정 (기업 수 * 400개 리뷰 최대값)
total_reviews = len(companies) * 400
progress_bar = tqdm(total=total_reviews, desc="Scraping Progress", unit="review")

# 각 기업별 크롤링
for company in companies:
    print(f"\n[+] Scraping reviews for {company}...")

    company_reviews = []
    page = 1

    while len(company_reviews) < 400:  # 최대 400개 리뷰 수집
        url = BASE_URL.format(company.lower(), page)

        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"[-] Request failed for {company} on page {page}: {e}")
            break

        soup = BeautifulSoup(response.text, "html.parser")

        # 리뷰 컨테이너 찾기
        reviews = soup.find_all("div", class_="company-reviews")
        if not reviews:
            print(f"[-] No more reviews found for {company} on page {page}")
            break

        for review in reviews:
            if len(company_reviews) >= 400:  # 400개 넘어가면 종료
                break

            # 직책 (Job Title)
            job_title = review.find("a", class_="header5")
            job_title = job_title.text.strip() if job_title else "Unknown"

            # 전체 평점 (Overall Rating)
            rating_stars = review.find("div", class_="rating large-star")
            if rating_stars:
                full_stars = len(rating_stars.find_all("span", class_="full"))
                half_stars = len(rating_stars.find_all("span", class_="half"))
                overall_rating = full_stars + (0.5 * half_stars)
            else:
                overall_rating = "Unknown"

            # 세부 항목 점수
            rating_details = {}
            categories = review.find_all("span", class_="span3 header9 text-right category")
            scores = review.find_all("span", class_="span1 header8 value")

            for cat, score in zip(categories, scores):
                category_name = cat.text.strip()
                rating_score = score.text.strip()
                rating_details[category_name] = rating_score

            # 데이터 추가
            company_reviews.append({
                "Company": company,
                "Job Title": job_title,
                "Overall Rating": overall_rating,
                **rating_details
            })

            # 진행률 바 업데이트
            progress_bar.update(1)

        page += 1
        sleep_time = random.uniform(1, 2)  # 1~2초 랜덤 대기
        print(f"Sleeping for {sleep_time:.2f} seconds before next request...")
        time.sleep(sleep_time)

    data.extend(company_reviews)

# 진행률 바 종료
progress_bar.close()

# DataFrame 생성
df = pd.DataFrame(data)

# CSV 저장
df.to_csv("careerbliss_reviews_fixed.csv", index=False, encoding="utf-8-sig")
print("\n✅ Scraping complete! Data saved to 'careerbliss_reviews_fixed.csv'")



Scraping Progress: 100%|██████████| 100/100 [03:59<00:00,  2.39s/review]



[+] Scraping reviews for Tesla...



Scraping Progress:   0%|          | 1/1200 [00:01<25:49,  1.29s/review]

Sleeping for 1.89 seconds before next request...



Scraping Progress:   1%|▏         | 17/1200 [00:04<04:25,  4.45review/s]

...

Sleeping for 1.07 seconds before next request...



Scraping Progress:  99%|█████████▉| 1185/1200 [03:31<00:02,  5.33review/s]

Sleeping for 1.72 seconds before next request...


Scraping Progress: 100%|██████████| 1200/1200 [03:33<00:00,  5.63review/s]


✅ Scraping complete! Data saved to 'careerbliss_reviews_fixed.csv'


## 날짜, 위치 포함

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm

# 크롤링할 기업 리스트
companies = [
    "Tesla", "Nvidia", "Apple"
]

# CareerBliss 리뷰 URL
BASE_URL = "https://www.careerbliss.com/{}/reviews/?page={}"

# User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 데이터 저장 리스트
data = []

# 총 작업량 설정
total_reviews = len(companies) * 400
progress_bar = tqdm(total=total_reviews, desc="Scraping Progress", unit="review")

# 크롤링 시작
for company in companies:
    print(f"\n[+] Scraping reviews for {company}...")

    company_reviews = []
    page = 1

    while len(company_reviews) < 400:
        url = BASE_URL.format(company.lower(), page)

        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"[-] Request failed for {company} on page {page}: {e}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        reviews = soup.find_all("div", class_="company-reviews")

        if not reviews:
            print(f"[-] No more reviews found for {company} on page {page}")
            break

        for review in reviews:
            if len(company_reviews) >= 400:
                break

            # 직책
            job_title = review.find("a", class_="header5")
            job_title = job_title.text.strip() if job_title else "Unknown"

            # 전체 평점
            rating_stars = review.find("div", class_="rating large-star")
            if rating_stars:
                full_stars = len(rating_stars.find_all("span", class_="full"))
                half_stars = len(rating_stars.find_all("span", class_="half"))
                overall_rating = full_stars + (0.5 * half_stars)
            else:
                overall_rating = "Unknown"

            # 세부 항목
            rating_details = {}
            categories = review.find_all("span", class_="span3 header9 text-right category")
            scores = review.find_all("span", class_="span1 header8 value")

            for cat, score in zip(categories, scores):
                category_name = cat.text.strip()
                rating_score = score.text.strip()
                rating_details[category_name] = rating_score

            # 날짜 및 위치 정보
            location_tag = review.find("span", class_="header13 spotlight-date")
            if location_tag:
                location_text = location_tag.get_text(strip=True)

                if "Posted" in location_text and "in" in location_text:
                    parts = location_text.replace("Posted", "").strip().split("in")
                    posted_date = parts[0].strip()
                    location = parts[1].strip()
                elif "in" in location_text:
                    posted_date = "Unknown"
                    location = location_text.split("in")[-1].strip()
                else:
                    posted_date = "Unknown"
                    location = "Unknown"
            else:
                posted_date = "Unknown"
                location = "Unknown"

            # 데이터 추가
            company_reviews.append({
                "Company": company,
                "Job Title": job_title,
                "Overall Rating": overall_rating,
                "Posted Date": posted_date,
                "Location": location,
                **rating_details
            })

            progress_bar.update(1)

        page += 1
        sleep_time = random.uniform(1, 2)
        print(f"Sleeping for {sleep_time:.2f} seconds...")
        time.sleep(sleep_time)

    data.extend(company_reviews)

# 진행률 바 종료
progress_bar.close()

# DataFrame 생성 및 저장
df = pd.DataFrame(data)
df.to_csv("careerbliss_location.csv", index=False, encoding="utf-8-sig")
print("\n✅ Scraping complete! Data saved to 'careerbliss_reviews_with_location.csv'")


Scraping Progress:   0%|          | 0/1200 [00:00<?, ?review/s]


[+] Scraping reviews for Tesla...


Scraping Progress:   0%|          | 1/1200 [00:01<32:27,  1.62s/review]

Sleeping for 1.36 seconds...


Scraping Progress:   1%|▏         | 17/1200 [00:04<04:19,  4.56review/s]

...

Sleeping for 1.66 seconds...


Scraping Progress:  99%|█████████▉| 1185/1200 [03:37<00:02,  6.45review/s]

Sleeping for 1.01 seconds...


Scraping Progress: 100%|██████████| 1200/1200 [03:38<00:00,  5.50review/s]


✅ Scraping complete! Data saved to 'careerbliss_reviews_with_location.csv'


## 날짜 수집 다시

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm

# 크롤링할 기업 리스트
companies = [
    "Tesla", "Nvidia", "Apple"
]

# CareerBliss 리뷰 URL
BASE_URL = "https://www.careerbliss.com/{}/reviews/?page={}"

# User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 데이터 저장 리스트
data = []

# 총 작업량 설정
total_reviews = len(companies) * 400
progress_bar = tqdm(total=total_reviews, desc="Scraping Progress", unit="review")

# 크롤링 시작
for company in companies:
    print(f"\n[+] Scraping reviews for {company}...")

    company_reviews = []
    page = 1

    while len(company_reviews) < 400:
        url = BASE_URL.format(company.lower(), page)

        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"[-] Request failed for {company} on page {page}: {e}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        reviews = soup.find_all("div", class_="company-reviews")

        if not reviews:
            print(f"[-] No more reviews found for {company} on page {page}")
            break

        for review in reviews:
            if len(company_reviews) >= 400:
                break

            # 직책
            job_title = review.find("a", class_="header5")
            job_title = job_title.text.strip() if job_title else "Unknown"

            # 전체 평점
            rating_stars = review.find("div", class_="rating large-star")
            if rating_stars:
                full_stars = len(rating_stars.find_all("span", class_="full"))
                half_stars = len(rating_stars.find_all("span", class_="half"))
                overall_rating = full_stars + (0.5 * half_stars)
            else:
                overall_rating = "Unknown"

            # 세부 항목
            rating_details = {}
            categories = review.find_all("span", class_="span3 header9 text-right category")
            scores = review.find_all("span", class_="span1 header8 value")

            for cat, score in zip(categories, scores):
                category_name = cat.text.strip()
                rating_score = score.text.strip()
                rating_details[category_name] = rating_score

            # 날짜 추출 (location은 삭제)
            date_tag = review.select_one("span.age span.ng-binding")
            posted_date = date_tag.text.strip() if date_tag else "Unknown"

            # 데이터 추가
            company_reviews.append({
                "Company": company,
                "Job Title": job_title,
                "Overall Rating": overall_rating,
                "Posted Date": posted_date,
                **rating_details  # rating_details 안에 세부 항목들 추가
            })

            progress_bar.update(1)

        page += 1
        sleep_time = random.uniform(1, 2)
        print(f"Sleeping for {sleep_time:.2f} seconds...")
        time.sleep(sleep_time)

    data.extend(company_reviews)

# 진행률 바 종료
progress_bar.close()

# DataFrame 생성 및 저장
df = pd.DataFrame(data)
df.to_csv("careerbliss_reviews.csv", index=False, encoding="utf-8-sig")
print("\n✅ Scraping complete! Data saved to 'careerbliss_reviews.csv'")


Scraping Progress:   0%|          | 0/1200 [00:00<?, ?review/s]


[+] Scraping reviews for Tesla...


Scraping Progress:   0%|          | 1/1200 [00:02<41:04,  2.06s/review]

Sleeping for 1.65 seconds...


Scraping Progress:   1%|▏         | 17/1200 [00:05<05:46,  3.42review/s]

...

Sleeping for 1.24 seconds...


Scraping Progress:  99%|█████████▉| 1185/1200 [03:52<00:02,  5.29review/s]

Sleeping for 1.50 seconds...


Scraping Progress: 100%|██████████| 1200/1200 [03:53<00:00,  5.13review/s]


✅ Scraping complete! Data saved to 'careerbliss_reviews.csv'


## 날짜 다시

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm

# 크롤링할 기업 리스트
companies = [
    "Tesla", "Nvidia", "Apple"
]

# CareerBliss 리뷰 URL
BASE_URL = "https://www.careerbliss.com/{}/reviews/?page={}"

# User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 데이터 저장 리스트
data = []

# 총 작업량 설정
total_reviews = len(companies) * 400
progress_bar = tqdm(total=total_reviews, desc="Scraping Progress", unit="review")

# 크롤링 시작
for company in companies:
    print(f"\n[+] Scraping reviews for {company}...")

    company_reviews = []
    page = 1

    while len(company_reviews) < 400:
        url = BASE_URL.format(company.lower(), page)

        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"[-] Request failed for {company} on page {page}: {e}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        reviews = soup.find_all("div", class_="company-reviews")

        if not reviews:
            print(f"[-] No more reviews found for {company} on page {page}")
            break

        for review in reviews:
            if len(company_reviews) >= 400:
                break

            # 직책
            job_title = review.find("a", class_="header5")
            job_title = job_title.text.strip() if job_title else "Unknown"

            # 전체 평점
            rating_stars = review.find("div", class_="rating large-star")
            if rating_stars:
                full_stars = len(rating_stars.find_all("span", class_="full"))
                half_stars = len(rating_stars.find_all("span", class_="half"))
                overall_rating = full_stars + (0.5 * half_stars)
            else:
                overall_rating = "Unknown"

            # 세부 항목
            rating_details = {}
            categories = review.find_all("span", class_="span3 header9 text-right category")
            scores = review.find_all("span", class_="span1 header8 value")

            for cat, score in zip(categories, scores):
                category_name = cat.text.strip()
                rating_score = score.text.strip()
                rating_details[category_name] = rating_score

            # 위치 추출
            location_tag = review.find("span", class_="header13")
            location = location_tag.find("a").text.strip() if location_tag and location_tag.find("a") else "Unknown"

            # 데이터 추가
            company_reviews.append({
                "Company": company,
                "Job Title": job_title,
                "Overall Rating": overall_rating,
                "Location": location,
                **rating_details
            })

            progress_bar.update(1)

        page += 1
        sleep_time = random.uniform(1, 2)
        print(f"Sleeping for {sleep_time:.2f} seconds...")
        time.sleep(sleep_time)

    data.extend(company_reviews)

# 진행률 바 종료
progress_bar.close()

# DataFrame 생성 및 저장
df = pd.DataFrame(data)
df.to_csv("careerbliss_reviews_with_location.csv", index=False, encoding="utf-8-sig")
print("\n✅ Scraping complete! Data saved to 'careerbliss_reviews_with_location.csv'")


Scraping Progress:   0%|          | 0/1200 [00:00<?, ?review/s]


[+] Scraping reviews for Tesla...


Scraping Progress:   0%|          | 1/1200 [00:01<22:32,  1.13s/review]

Sleeping for 1.35 seconds...


Scraping Progress:   1%|▏         | 17/1200 [00:03<04:15,  4.64review/s]

...

Sleeping for 1.70 seconds...


Scraping Progress:  99%|█████████▉| 1185/1200 [03:39<00:02,  5.39review/s]

Sleeping for 1.23 seconds...


Scraping Progress: 100%|██████████| 1200/1200 [03:40<00:00,  5.44review/s]


✅ Scraping complete! Data saved to 'careerbliss_reviews_with_location.csv'
